In [ ]:
from audio_extractor.audio_processor import AudioExtractor, AudioChunker, VideoToTextPipeline

video_path = "/Users/youssefjanjar/Documents/formascience/volume/cours_1.mp4"
output_path = "/Users/youssefjanjar/Documents/formascience/volume/transcripts"



audio_extractor = AudioExtractor()
audio_extractor.extract_audio(video_path, output_path)

2025-07-27 15:48:19 - audio_extractor.audio_processor - INFO - Extracting audio from /Users/youssefjanjar/Documents/formascience/volume/cours_1.mp4
2025-07-27 15:48:23 - audio_extractor.audio_processor - INFO - Audio extracted successfully: /Users/youssefjanjar/Documents/formascience/volume/transcripts/cours_1_audio.wav


'/Users/youssefjanjar/Documents/formascience/volume/transcripts/cours_1_audio.wav'

In [2]:
from audio_extractor.audio_processor import AudioChunker
import os

audio_chunker = AudioChunker()
audio_path = "/Users/youssefjanjar/Documents/formascience/volume/audio/cours_1_audio.wav"
output_path = "/Users/youssefjanjar/Documents/formascience/volume/transcripts"

# Initialize chunker
chunker = AudioChunker()

# Split audio into 3-minute chunks
# This creates a folder named "audio_filename_chunks" automatically
chunk_paths = chunker.split_audio(
    audio_path, 
    chunk_duration_minutes=3.0  # 3-minute chunks
)

print(f"Created {len(chunk_paths)} chunks")
print(f"Chunks saved in: {os.path.dirname(chunk_paths[0]) if chunk_paths else 'N/A'}")

# Show first few chunk paths
for i, chunk in enumerate(chunk_paths[:3], 1):
    print(f"  Chunk {i}: {os.path.basename(chunk)}")








2025-07-27 15:59:51 - audio_extractor.audio_processor - INFO - Splitting audio into 3.0 minute chunks
2025-07-27 15:59:51 - audio_extractor.audio_processor - INFO - Output directory: /Users/youssefjanjar/Documents/formascience/volume/audio/cours_1_audio_chunks
2025-07-27 15:59:52 - audio_extractor.audio_processor - INFO - Created 25 audio chunks


Created 25 chunks
Chunks saved in: /Users/youssefjanjar/Documents/formascience/volume/audio/cours_1_audio_chunks
  Chunk 1: cours_1_audio_chunk_001.wav
  Chunk 2: cours_1_audio_chunk_002.wav
  Chunk 3: cours_1_audio_chunk_003.wav


In [ ]:
from transcription.whisper_client import WhisperTranscriber
prompt = """
Transcription académique d'un cours de médecine universitaire, en français.
Le locuteur est un professeur qui commente des diapositives.
Transcrivez uniquement le discours du professeur, en bon français, sans ajouter de descriptions de diapositives.
Ignorez les répétitions ou hésitations.
Privilégiez la clarté.
"""
timestamp_granularities = ["segment"]










# Transcription with timestamps

In [2]:
# Your specific use case - French academic transcription
from audio_extractor import WhisperTranscriber

# Academic French transcription prompt
prompt = """
Transcription académique d'un cours de médecine universitaire, en français.
Le locuteur est un professeur qui commente des diapositives.
Utilisez la terminologie médicale appropriée.
Indiquez les pauses importantes par des points.
Privilégiez la clarté.
"""

# Your audio file path
audio_path = "/Users/youssefjanjar/Documents/formascience/volume/audio/cours_1_audio_chunks/cours_1_audio_chunk_001.wav"

from audio_extractor import WhisperTranscriber

transcriber = WhisperTranscriber()

# French academic transcription with timestamp parsing
result = transcriber.transcribe_with_timestamps(
    audio_path=audio_path,
    language="fr",
    prompt=prompt,
    timestamp_granularities=["segment"]
)

print(f"✅ JSON saved to: {result['json_file']}")
print(f"✅ Parsed timestamps saved to: {result['parsed_file']}")

2025-07-27 16:52:05 - audio_extractor.transcription.whisper_client - INFO - Transcribing with timestamps: /Users/youssefjanjar/Documents/formascience/volume/audio/cours_1_audio_chunks/cours_1_audio_chunk_001.wav
2025-07-27 16:52:17 - audio_extractor.transcription.whisper_client - INFO - Transcription saved: /Users/youssefjanjar/Documents/formascience/volume/transcripts/json/cours_1_audio_chunk_001_transcription.json
2025-07-27 16:52:17 - audio_extractor.transcription.whisper_client - INFO - JSON transcription saved to: /Users/youssefjanjar/Documents/formascience/volume/transcripts/json/cours_1_audio_chunk_001_transcription.json
2025-07-27 16:52:17 - audio_extractor.transcription.whisper_client - INFO - Parsed timestamps saved to: /Users/youssefjanjar/Documents/formascience/volume/transcripts/parsed/cours_1_audio_chunk_001_timestamps.txt


✅ JSON saved to: /Users/youssefjanjar/Documents/formascience/volume/transcripts/json/cours_1_audio_chunk_001_transcription.json
✅ Parsed timestamps saved to: /Users/youssefjanjar/Documents/formascience/volume/transcripts/parsed/cours_1_audio_chunk_001_timestamps.txt


# PDF extraction


In [5]:
from pdf_extractor.pdf_processor import PDFProcessor
from pdf_extractor.slide_parser import SlideParser

pdf_path = "/Users/youssefjanjar/Documents/class_parser/volume/slides/cours_1.pdf"

"""Example of extracting and parsing PDF slides."""

# Initialize processor and parser
processor = PDFProcessor(output_dir="/Users/youssefjanjar/Documents/class_parser/output")
parser = SlideParser()


try:
    # Extract PDF metadata
    print("Extracting PDF metadata...")
    metadata = processor.extract_pdf_metadata(pdf_path)
    print(f"Title: {metadata.title}")
    print(f"Author: {metadata.author}")
    print(f"Pages: {metadata.page_count}")
    
    # Extract slide content
    print("\nExtracting slide content...")
    slides = processor.extract_slide_content(
        pdf_path=pdf_path,
        extract_images=True,
        convert_to_images=True,
        dpi=300
    )
    
    # Get extraction summary
    summary = processor.get_extraction_summary(slides)
    print(f"\nExtraction Summary:")
    print(f"- Total slides: {summary['total_slides']}")
    print(f"- Total text: {summary['total_text_length']:,} characters")
    print(f"- Total images: {summary['total_images']}")
    
    # Parse slides
    print("\nParsing slide structure...")
    parsed_slides = parser.parse_slides(slides)
    
    # Analyze presentation structure
    structure = parser.analyze_presentation_structure(parsed_slides)
    
    # Generate summary
    presentation_summary = parser.generate_presentation_summary(structure)
    print(f"\n{presentation_summary}")
    
    # Show detailed analysis for top 3 most important slides
    print("\nTop 3 Most Important Slides:")
    print("=" * 40)
    
    top_slides = sorted(parsed_slides, key=lambda x: x.importance_score, reverse=True)[:3]
    
    for slide in top_slides:
        print(f"\nSlide {slide.page_number} - {slide.slide_type.upper()}")
        print(f"Importance Score: {slide.importance_score:.2f}")
        
        if slide.title:
            print(f"Title: {slide.title}")
        
        if slide.bullet_points:
            print(f"Bullet Points:")
            for bp in slide.bullet_points[:3]:
                print(f"  • {bp}")
        
        if slide.keywords:
            print(f"Keywords: {', '.join(slide.keywords[:5])}")
        
        print("-" * 30)
    
    # Save extracted text
    processor.save_extracted_text(slides, "extracted_presentation.txt")
    print(f"\nExtracted text saved to: example_output/text/extracted_presentation.txt")
    
except FileNotFoundError:
    print(f"PDF file not found: {pdf_path}")
    print("Please place a PDF file named 'example_presentation.pdf' in the current directory")
except Exception as e:
    print(f"Error processing PDF: {e}")

Extracting PDF metadata...
Title: Présentation PowerPoint
Author: Jean Muller
Pages: 111

Extracting slide content...

Extraction Summary:
- Total slides: 111
- Total text: 31,988 characters
- Total images: 591

Parsing slide structure...

Presentation: L1SpS: UE 2 Les molécules du vivant
Introduction
3
Total slides: 111

Main Topics:
1. L1SpS: UE 2 Les molécules du vivant
7.
2. L1SpS: UE 2 Les molécules du vivant
3. L1SpS: UE 2 Les molécules du vivant
4. L1SpS: UE 2 Les molécules du vivant
5. L1SpS: UE 2 Les molécules du vivant

Key terms: molécules, vivant, l1sps, génome, gènes, gène, humain, génomes, répétés, variation

Slide breakdown: 110 content slides, 0 summary slides

Top 3 Most Important Slides:

Slide 1 - TEXT_HEAVY
Importance Score: 1.00
Title: L1SpS: UE 2 Les molécules du vivant
7.
Bullet Points:
  • 1 Architecture du génome
  • E.2 Les molécules du vivant
  • Organisation du génome humain, méthodes en
Keywords: molécules, vivant, génome, humain, laboratoire
--------------